In [5]:
from pymongo import MongoClient
import requests

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017")
db = client['polytest']
markets_col = db['markets']

url = "https://gamma-api.polymarket.com/markets"
limit = 50
offset = 0
total_stored = 0


In [6]:
while True:
    params = {
        "limit": limit,
        "offset": offset,
        "active": True,
        "start_date_min": "2025-12-01T00:00:00Z",
        "start_date_max": "2025-12-31T23:59:59Z",
        "order": "updatedAt",
        "ascending": False
    }
    markets = requests.get(url, params=params, timeout=10).json()
    
    if not markets:
        break

    for m in markets:
        markets_col.update_one(
            {"id": m["id"]},
            {"$set": m},
            upsert=True
        )
    
    total_stored += len(markets)
    offset += len(markets)

print(f"Stored {total_stored} markets from 2025")

KeyboardInterrupt: 